In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings; 
warnings.simplefilter('ignore')

In [13]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [14]:
!git clone https://github.com/farelarden/Dicoding-Capstone-ML.git

fatal: destination path 'Dicoding-Capstone-ML' already exists and is not an empty directory.


In [15]:
df_Makanan = pd.read_csv('/content/Dicoding-Capstone-ML/Dataset/Fix Dataset/fix 02 - Makanan.csv')
df_Minuman = pd.read_csv('/content/Dicoding-Capstone-ML/Dataset/Fix Dataset/fix 02 - Minuman.csv')
df_Rumah = pd.read_csv('/content/Dicoding-Capstone-ML/Dataset/Fix Dataset/fix 03 - Rumah.csv')
df_Sandang = pd.read_csv('/content/Dicoding-Capstone-ML/Dataset/Fix Dataset/fix 04 - Sandang.csv')
df_Transportasi = pd.read_csv('/content/Dicoding-Capstone-ML/Dataset/Fix Dataset/fix 07 - Transportasi.csv')

In [16]:
df_Makanan.head()

,Unnamed: 0,URUTAN,BULAN,KOTA MEULABOH,KOTA BANDA ACEH,KOTA LHOKSEUMAWE,KOTA SIBOLGA,KOTA PEMATANG SIANTAR,KOTA MEDAN,KOTA PADANGSIDIMPUAN,KOTA PADANG,KOTA BUKITTINGGI,TEMBILAHAN,KOTA PEKANBARU,KOTA DUMAI,BUNGO,KOTA JAMBI,KOTA PALEMBANG,KOTA LUBUKLINGGAU,KOTA BENGKULU,KOTA BANDAR LAMPUNG,KOTA METRO,TANJUNG PANDAN,KOTA PANGKAL PINANG,KOTA BATAM,KOTA TANJUNG PINANG,DKI JAKARTA,KOTA BOGOR,KOTA SUKABUMI,KOTA BANDUNG,KOTA CIREBON,KOTA BEKASI,KOTA DEPOK,KOTA TASIKMALAYA,CILACAP,PURWOKERTO,KUDUS,KOTA SURAKARTA,KOTA SEMARANG,KOTA TEGAL,...,KOTA MALANG,KOTA PROBOLINGGO,KOTA MADIUN,KOTA SURABAYA,KOTA TANGERANG,KOTA CILEGON,KOTA SERANG,SINGARAJA,KOTA DENPASAR,KOTA MATARAM,KOTA BIMA,MAUMERE,KOTA KUPANG,KOTA PONTIANAK,KOTA SINGKAWANG,SAMPIT,KOTA PALANGKA RAYA,TANJUNG,KOTA BANJARMASIN,KOTA BALIKPAPAN,KOTA SAMARINDA,KOTA TARAKAN,KOTA MANADO,KOTA PALU,BULUKUMBA,WATAMPONE,KOTA MAKASSAR,KOTA PARE-PARE,KOTA PALOPO,KOTA KENDARI,KOTA BAU-BAU,KOTA GORONTALO,MAMUJU,KOTA AMBON,KOTA TUAL,KOTA TERNATE,MANOKWARI,KOTA SORONG,MERAUKE,KOTA JAYAPURA
0,0,0,2014-01-01,1.10,0.25,0.31,0.20,0.12,0.39,1.53,0.25,0.12,0.0,1.34,0.66,0.68,0.34,0.37,0.04,0.56,0.37,-0.06,0.00,2.65,0.03,1.08,0.66,0.73,3.43,1.31,0.00,1.58,0.14,0.17,0.00,1.16,0.54,0.85,0.44,0.91,...,0.36,0.06,1.29,1.54,1.07,0.45,0.57,1.47,1.17,0.01,7.81,0.20,0.08,0.64,0.18,2.49,2.01,0.00,0.20,1.09,0.24,0.95,0.51,0.01,0.33,0.33,2.39,2.33,0.67,0.11,0.65,0.70,0.06,0.00,1.33,0.00,0.30,2.26,1.89,0.76
1,1,1,2014-02-01,0.88,0.00,0.00,0.29,0.79,0.22,0.17,0.11,0.63,0.0,1.02,0.80,0.93,0.36,0.00,0.00,0.00,0.26,0.00,0.00,0.02,0.00,0.18,0.01,0.36,0.37,-0.06,0.27,0.09,0.03,0.09,0.58,0.04,1.67,0.52,0.14,1.58,...,3.19,0.77,0.09,0.25,0.31,1.15,1.15,0.00,0.87,1.10,0.06,0.43,1.05,0.06,2.37,0.27,2.12,1.31,2.68,0.01,1.02,0.12,0.97,1.99,0.19,1.00,-0.05,0.49,0.08,0.35,0.61,3.72,0.00,0.00,1.76,1.05,1.21,0.80,0.26,0.38
2,2,2,2014-03-01,1.42,0.10,0.73,0.29,0.16,0.22,0.00,0.01,0.00,0.0,0.66,0.02,0.00,0.97,0.31,0.00,0.01,0.06,0.00,1.02,-0.01,0.05,0.69,0.46,0.02,0.52,0.03,0.15,0.33,0.08,0.46,0.04,0.07,0.00,0.31,0.57,0.15,...,0.27,1.25,-0.05,0.64,1.21,0.34,1.98,1.66,0.09,0.00,1.69,0.28,0.02,0.00,0.02,2.09,0.00,0.97,0.05,-0.01,0.17,0.28,0.44,1.59,0.10,0.55,1.12,0.02,0.10,2.53,0.00,0.00,0.41,0.00,0.00,0.05,0.53,0.00,0.24,0.39
3,3,3,2014-04-01,0.17,0.04,1.26,0.12,0.67,1.31,0.93,0.36,0.18,0.0,0.23,0.11,0.17,0.36,0.07,0.00,0.76,0.00,-0.01,0.00,0.00,0.09,0.03,0.04,0.12,0.43,0.71,0.00,0.38,0.18,0.41,0.43,0.17,0.89,0.10,0.06,0.32,...,0.05,-0.01,0.42,1.37,1.06,0.14,0.00,3.20,0.40,0.09,0.00,0.04,0.05,-0.04,0.03,1.42,0.20,0.83,0.02,0.10,0.36,0.01,1.16,0.02,0.15,0.41,1.18,0.25,0.00,0.10,0.07,0.26,0.04,1.81,0.20,0.00,2.59,0.00,0.81,0.41
4,4,4,2014-05-01,-0.01,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.40,0.0,0.04,0.00,0.00,0.17,0.00,2.58,0.00,0.00,0.06,0.00,0.02,1.20,0.22,0.17,0.20,0.54,0.61,-0.06,0.48,0.01,0.12,0.91,0.00,0.26,0.02,0.18,0.00,...,0.93,0.24,0.17,0.09,0.19,0.00,0.45,1.85,0.73,1.28,0.46,0.00,0.10,0.31,0.45,0.19,0.00,0.52,0.00,0.09,1.08,0.04,0.81,0.00,0.75,0.42,0.27,1.07,1.03,1.26,0.68,0.00,0.04,0.46,0.00,0.15,0.62,0.00,0.07,0.27


In [17]:
time = df_Makanan['Unnamed: 0']
inflation = df_Makanan['KOTA MEULABOH']

In [18]:
inflation = np.array(inflation)
time = np.array(time)

In [19]:
plt.figure(figsize=(10, 6))
plot_series(time, inflation)

In [20]:
split_time = 2612
time_train = time[:split_time]
x_train = inflation[:split_time]
time_valid = time[split_time:]
x_valid = inflation[split_time:]

window_size = 30
batch_size = 32

In [21]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [22]:
import tensorflow as tf
train_set = windowed_dataset(x_train, window_size=60, batch_size=100, shuffle_buffer=1000)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True), #declaring the LSTM layers
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

In [23]:
optimizer=tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set,epochs=100)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 0.5396 - mae: 0.8955
Epoch 2/100
1/1 [==============================] - 0s 86ms/step - loss: 8.7593 - mae: 9.2593
Epoch 3/100
1/1 [==============================] - 0s 93ms/step - loss: 6.4075 - mae: 6.9075
Epoch 4/100
1/1 [==============================] - 0s 85ms/step - loss: 1.1464 - mae: 1.5773
Epoch 5/100
1/1 [==============================] - 0s 83ms/step - loss: 2.2644 - mae: 2.7603
Epoch 6/100
1/1 [==============================] - 0s 85ms/step - loss: 9.4597 - mae: 9.9597
Epoch 7/100
1/1 [==============================] - 0s 85ms/step - loss: 3.5755 - mae: 4.0755
Epoch 8/100
1/1 [==============================] - 0s 84ms/step - loss: 18.2057 - mae: 18.7057
Epoch 9/100
1/1 [==============================] - 0s 86ms/step - loss: 20.7706 - mae: 21.2706
Epoch 10/100
1/1 [==============================] - 0s 83ms/step - loss: 4.9302 - mae: 5.4302
Epoch 11/100
1/1 [==============================] - 0s 86ms/step - 